<a href="https://colab.research.google.com/github/morim3/DeepKalmanFilter/blob/master/my_experiment/pqc_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/morim3/qiskit-optimization
!nvidia-smi
# !pip install -r qiskit-optimization/requirements-dev.txt
!pip install qiskit-aer-gpu
!pip install docplex
#test
import sys
sys.path.append("./qiskit-optimization/")

Cloning into 'qiskit-optimization'...
remote: Enumerating objects: 4042, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 4042 (delta 45), reused 66 (delta 24), pack-reused 3935
Receiving objects: 100% (4042/4042), 5.15 MiB | 17.95 MiB/s, done.
Resolving deltas: 100% (2658/2658), done.
Thu Dec  1 07:02:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Of

In [ ]:
%load_ext autoreload
%autoreload

from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit_optimization.algorithms import PBILGroverOptimizer, GroverOptimizer, MinimumEigenOptimizer
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
from qiskit import Aer
from docplex.mp.model import Model
import numpy as np
print(Aer.backends())
backend = Aer.get_backend('aer_simulator')
backend.set_options(device='GPU')
print(backend)

[AerSimulator('aer_simulator'), AerSimulator('aer_simulator_statevector'), AerSimulator('aer_simulator_statevector_gpu'), AerSimulator('aer_simulator_density_matrix'), AerSimulator('aer_simulator_density_matrix_gpu'), AerSimulator('aer_simulator_stabilizer'), AerSimulator('aer_simulator_matrix_product_state'), AerSimulator('aer_simulator_extended_stabilizer'), AerSimulator('aer_simulator_unitary'), AerSimulator('aer_simulator_unitary_gpu'), AerSimulator('aer_simulator_superop'), QasmSimulator('qasm_simulator'), StatevectorSimulator('statevector_simulator'), UnitarySimulator('unitary_simulator'), PulseSimulator('pulse_simulator')]
aer_simulator_gpu


In [ ]:
model = Model()
xs = [model.binary_var(name=f"x{i}") for i in range(15)]
objective = sum([np.random.randint(0, 3) * x for x in xs]) + sum([np.random.randint(-2, 3) * x * y for x in xs for y in xs])
# objective = (-xs[0]+2*xs[1]-3*xs[2]-2*xs[0]*xs[2]-1*xs[1]*xs[2])
model.minimize(objective)
qp = from_docplex_mp(model)
print(qp.prettyprint())

Problem name: docplex_model1

Minimize
  -2*x0^2 + x0*x1 - x0*x11 + x0*x12 - 2*x0*x14 + 2*x0*x2 + x0*x3 - x0*x4
  - 2*x0*x5 - 3*x0*x6 - 4*x0*x8 + x0*x9 - 2*x1^2 + x1*x10 - 3*x1*x11 + x1*x13
  + x1*x2 - 3*x1*x3 + 2*x1*x4 - 2*x1*x5 - 3*x1*x6 - 3*x1*x7 - 2*x1*x8 + 2*x1*x9
  + 2*x10^2 - 3*x10*x11 - 4*x10*x12 + x10*x13 - x10*x14 + 2*x11^2 + 3*x11*x12
  - x11*x14 + x12^2 + 3*x12*x13 - x12*x14 + x13^2 - x13*x14 - 2*x14^2 - x2*x11
  - x2*x12 + x2*x14 - x2^2 + 2*x2*x3 - 3*x2*x4 + 3*x2*x5 - x2*x6 + 4*x2*x7
  + 2*x2*x8 - 2*x2*x9 + x3*x10 - 2*x3*x11 + x3*x12 + 4*x3*x13 - 2*x3*x14
  - 2*x3*x4 + 2*x3*x5 - x3*x6 + 2*x3*x7 - x3*x8 - 2*x3*x9 - 2*x4*x10 + x4*x11
  - x4*x12 + 4*x4*x13 - 2*x4*x14 + x4^2 - 2*x4*x5 - 2*x4*x6 - 2*x4*x7 + 2*x4*x9
  + 2*x5*x10 + x5*x13 + x5*x14 + 2*x5^2 - 2*x5*x6 + x5*x8 - x5*x9 + x6*x10
  - x6*x11 - x6*x12 + 2*x6*x13 - 3*x6*x14 - x6^2 + 2*x6*x7 + x6*x8 + 3*x6*x9
  + 4*x7*x10 + 3*x7*x11 - x7*x12 + 3*x7*x13 + 2*x7*x14 - 2*x7^2 + x7*x8
  - 3*x7*x9 - 2*x8*x10 + x8*x11 + 3*x8*x12 

In [ ]:
exact_solver = MinimumEigenOptimizer(NumPyMinimumEigensolver())
exact_result = exact_solver.solve(qp)
print(exact_result.prettyprint())

objective function value: -31.0
variable values: x0=1.0, x1=1.0, x2=0.0, x3=1.0, x4=1.0, x5=0.0, x6=1.0, x7=0.0, x8=1.0, x9=0.0, x10=1.0, x11=1.0, x12=0.0, x13=0.0, x14=1.0
status: SUCCESS


In [ ]:
grover_optimizer = PBILGroverOptimizer(8, num_iterations=15, quantum_instance=backend, 
                                       pbil_sample_num=1, quantile=0., lr=0.05, )
results = grover_optimizer.solve(qp)
print(results.prettyprint())
print(results.operation_counts)

sampled [0 1 1 1 1 0 0 0 1 0 0 0 1 0 1]
value -3
real_val -3.0
param [0.475 0.525 0.525 0.525 0.525 0.475 0.475 0.475 0.525 0.475 0.475 0.475
 0.525 0.475 0.525]
threshold -3
sampled [1 0 1 1 1 1 0 1 0 0 1 1 0 1 0]
value 26
real_val 26.0
sampled [0 0 1 1 1 0 0 0 1 1 1 0 1 1 1]
value 0
real_val 0.0
sampled [1 1 0 0 0 0 0 0 1 1 1 0 0 1 1]
value -4
real_val -4.0
param [0.50125 0.54875 0.49875 0.49875 0.49875 0.45125 0.45125 0.45125 0.54875
 0.50125 0.50125 0.45125 0.49875 0.50125 0.54875]
threshold -4
sampled [1 0 1 1 1 0 0 1 0 0 0 0 0 0 1]
value -3
real_val -3.0
sampled [0 1 1 0 0 0 1 0 1 1 1 1 1 0 1]
value -6
real_val -6.0
param [0.4761875 0.5713125 0.5238125 0.4738125 0.4738125 0.4286875 0.4786875
 0.4286875 0.5713125 0.5261875 0.5261875 0.4786875 0.5238125 0.4761875
 0.5713125]
threshold -6
sampled [0 0 0 0 1 0 0 0 1 1 0 1 0 0 0]
value 4
real_val 4.0
sampled [0 1 1 1 1 0 1 1 1 0 1 0 1 0 1]
value -9
real_val -9.0
param [0.45237812 0.59274688 0.54762188 0.50012187 0.50012187 0.40725312


In [ ]:
def eval_num(operation_dict):
    eval_num = 0
    for x in operation_dict.values():
        eval_num += x['Q(x)']
        if 'Q' in x:
            eval_num += x['Q']
    return eval_num

In [ ]:
eval_num(results.operation_counts)

91

In [ ]:
grover_optimizer = GroverOptimizer(8, num_iterations=15, quantum_instance=backend,)
results = grover_optimizer.solve(qp)
print(results.prettyprint())
print(results.operation_counts)

objective function value: -26.0
variable values: x0=1.0, x1=1.0, x2=1.0, x3=1.0, x4=1.0, x5=0.0, x6=1.0, x7=0.0, x8=1.0, x9=0.0, x10=0.0, x11=1.0, x12=0.0, x13=0.0, x14=1.0
status: SUCCESS
{0: OrderedDict([('measure', 24), ('h', 15), ('Q(x)', 1)]), 1: OrderedDict([('measure', 24), ('h', 15), ('Q(x)', 1)]), 2: OrderedDict([('measure', 24), ('h', 15), ('Q(x)', 1)]), 3: OrderedDict([('measure', 24), ('h', 15), ('Q', 2), ('Q(x)', 1)]), 4: OrderedDict([('measure', 24), ('h', 15), ('Q', 3), ('Q(x)', 1)]), 5: OrderedDict([('measure', 24), ('h', 15), ('Q(x)', 1)]), 6: OrderedDict([('measure', 24), ('h', 15), ('Q(x)', 1), ('Q', 1)]), 7: OrderedDict([('measure', 24), ('h', 15), ('Q(x)', 1)]), 8: OrderedDict([('measure', 24), ('h', 15), ('Q(x)', 1), ('Q', 1)]), 9: OrderedDict([('measure', 24), ('h', 15), ('Q(x)', 1)]), 10: OrderedDict([('measure', 24), ('h', 15), ('Q', 3), ('Q(x)', 1)]), 11: OrderedDict([('measure', 24), ('h', 15), ('Q(x)', 1), ('Q', 1)]), 12: OrderedDict([('measure', 24), ('h', 

In [ ]:
eval_num(results.operation_counts)

119